In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 2.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=29c02271b8acb6e2c4234f1a99ae91895c92c2e291ce9de465a2cfc14c1a8919
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=646e350870e452aabf5d004fc3aac632ea236766a801cf9db18c762dc460bd8b
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import tensorflow as tf
import keras
from keras import models   
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Dropout,BatchNormalization
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from keras.preprocessing import image
from kerastuner.tuners import RandomSearch
from kerastuner import HyperModel
import matplotlib.pyplot as plt
import datetime
import numpy as np
import cv2
import os
import time
import timeit

In [4]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        width_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.2,1.2],
        fill_mode='constant',cval=255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
training_set = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/train',
        target_size=(150,150),
        batch_size=32,
        class_mode='binary')
test_set = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/test',
        target_size=(150,150),
        batch_size=32,
        class_mode='binary')

Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [6]:
X_train,Y_train = training_set.next()
X_test,Y_test = test_set.next()

In [7]:
X_train.shape

(32, 150, 150, 3)

In [33]:
# BUILDING MODEL TO CLASSIFY BETWEEN MASK AND NO MASK
model=Sequential()
model.add(Conv2D(16,(3,3),activation='relu',padding='same',input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)) )
model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)) )
model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)) )
model.add(Flatten())
model.add(Dense(128,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform()))
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform()))
model.add(Dropout(0.25))
model.add(Dense(1,activation='sigmoid'))





In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 150, 150, 16)      448       
_________________________________________________________________
batch_normalization_2 (Batch (None, 150, 150, 16)      64        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 75, 75, 32)        4640      
_________________________________________________________________
batch_normalization_3 (Batch (None, 75, 75, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 37, 37, 32)       

In [11]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [12]:
start = timeit.default_timer()
batch_no=32
model_saved=model.fit(
        training_set,
        batch_size=batch_no,
        epochs=20,
        verbose=1,
        validation_data=test_set,
        )
end = timeit.default_timer()
print("Time Taken to run the model:",end - start, "seconds")
model.save('mymodel.h5',model_saved)


Epoch 1/20
42/42 [==============================] - 311s 7s/step - loss: 1.4803 - accuracy: 0.6222 - val_loss: 0.5580 - val_accuracy: 0.9330
Epoch 2/20
42/42 [==============================] - 47s 1s/step - loss: 0.3513 - accuracy: 0.8564 - val_loss: 0.4396 - val_accuracy: 0.8196
Epoch 3/20
42/42 [==============================] - 47s 1s/step - loss: 0.2556 - accuracy: 0.9111 - val_loss: 0.3095 - val_accuracy: 0.9278
Epoch 4/20
42/42 [==============================] - 47s 1s/step - loss: 0.2101 - accuracy: 0.9239 - val_loss: 0.3543 - val_accuracy: 0.8247
Epoch 5/20
42/42 [==============================] - 47s 1s/step - loss: 0.1946 - accuracy: 0.9201 - val_loss: 0.3579 - val_accuracy: 0.8093
Epoch 6/20
42/42 [==============================] - 47s 1s/step - loss: 0.2012 - accuracy: 0.9110 - val_loss: 1.0145 - val_accuracy: 0.5103
Epoch 7/20
42/42 [==============================] - 47s 1s/step - loss: 0.4251 - accuracy: 0.8714 - val_loss: 0.2082 - val_accuracy: 0.8969
Epoch 8/20
42/42 [=

In [13]:

class CNNHyperModel(HyperModel):

    def __init__(self):
        self.input_shape = (150, 150, 3)
        self.num_classes = 2

    def build(self, hp):

        model_ht = keras.Sequential()

        
        model_ht.add(
            Conv2D(
                filters=hp.Choice(
                    'num_filters',
                    values=[16, 32],
                    default=32,
                ),
                activation='relu',
                padding='same',
                kernel_size=3
            )
        )

        model_ht.add(BatchNormalization())

        model_ht.add(MaxPooling2D(pool_size=2))

        
        model_ht.add(
            Conv2D(
                filters=hp.Choice(
                    'num_filters',
                    values=[16, 32],
                    default=32,
                ),
                activation='relu',
                padding='same',
                kernel_size=3
            )
        )
        model_ht.add(BatchNormalization())

        model_ht.add(MaxPooling2D(pool_size=2))
  
        

        model_ht.add(
            Conv2D(
                filters=hp.Choice(
                    'num_filters',
                    values=[64, 32],
                    default=64,
                ),
                activation='relu',
                padding='same',
                kernel_size=3
            )
        )

        model_ht.add(MaxPooling2D(pool_size=2))

        model_ht.add(Flatten())

        model_ht.add(
            Dense(
                units=hp.Int(
                    'units',
                    min_value=32,
                    max_value=128,
                    step=32,
                    default=128
                ),
                activation='relu',
                kernel_initializer=tf.keras.initializers.HeUniform()
            )
        )

        
        model_ht.add(
            Dropout(
                rate=hp.Float(
                    'dropout_1',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        model_ht.add(
            Dense(
                units=hp.Int(
                    'units',
                    min_value=32,
                    max_value=128,
                    step=32,
                    default=128
                ),
                activation='relu',
                kernel_initializer=tf.keras.initializers.HeUniform()
            )
        )

        
        model_ht.add(
            Dropout(
                rate=hp.Float(
                    'dropout_1',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )

        model_ht.add(Dense(1, activation='sigmoid'))

        model_ht.compile(
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        return model_ht

hypermodel = CNNHyperModel()


In [14]:
from kerastuner.tuners import RandomSearch
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    project_name='face-mask'
)

In [15]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_filters (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32], 'ordered': True}
units (Int)
{'default': 128, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': None}
dropout_1 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
learning_rate (Float)
{'default': 0.001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [17]:
tuner.search(X_train, Y_train, epochs=5, validation_data=(X_test, Y_test),batch_size = 32)

Trial 10 Complete [00h 07m 39s]
val_accuracy: 0.9458763003349304

Best val_accuracy So Far: 0.9458763003349304
Total elapsed time: 00h 51m 05s
INFO:tensorflow:Oracle triggered exit


In [42]:
tuner.results_summary()

# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]
tf.train.Checkpoint.restore
loss, accuracy = best_model.evaluate(X_test, Y_test)
model.save('hyper_model2.h5',best_model)

Results summary
Results in ./face-mask
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_filters: 32
units: 64
dropout_1: 0.1
learning_rate: 0.007446200469868547
Score: 0.9458763003349304
Trial summary
Hyperparameters:
num_filters: 16
units: 96
dropout_1: 0.35000000000000003
learning_rate: 0.0028102636841753114
Score: 0.9407216608524323
Trial summary
Hyperparameters:
num_filters: 32
units: 96
dropout_1: 0.30000000000000004
learning_rate: 0.0012440718000666692
Score: 0.938144326210022
Trial summary
Hyperparameters:
num_filters: 16
units: 96
dropout_1: 0.4
learning_rate: 0.0009460923621506966
Score: 0.9175257980823517
Trial summary
Hyperparameters:
num_filters: 16
units: 64
dropout_1: 0.05
learning_rate: 0.0064523261336763725
Score: 0.9149484634399414
Trial summary
Hyperparameters:
num_filters: 16
units: 96
dropout_1: 0.35000000000000003
learning_rate: 0.00048690017276281297
Score: 0.9123711585998535
Trial summary
Hyperparameters:
n

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 150, 150, 16)      448       
_________________________________________________________________
batch_normalization_2 (Batch (None, 150, 150, 16)      64        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 75, 75, 32)        4640      
_________________________________________________________________
batch_normalization_3 (Batch (None, 75, 75, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 37, 37, 32)       

In [43]:
# FOR REAL TIME DETECTION
mymodel=load_model('hyper_model2.h5')
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('/content/drive/MyDrive/haarcascade_frontalface_default.xml')
while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict_classes(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
